In [1]:
import pandas as pd
import warnings
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
df_industries = pd.read_excel('/Users/ju/Projects/00_SMU/mqf_practice/QF600_Asset_Pricing/Exam/data/Exam-Industry.xlsx', index_col='Date')
df_market = pd.read_excel('/Users/ju/Projects/00_SMU/mqf_practice/QF600_Asset_Pricing/Exam/data/Exam–Market.xlsx', index_col='Date')
df = pd.merge(df_industries, df_market, left_index=True, right_index=True)
# df_risk_factors = pd.read_excel('./data/Risk_Factors.xlsx', index_col='Date')

In [3]:
# Perfrom regression
excess_returns = df.values
market_returns = df_market.values

In [4]:
sharpe        = np.mean(excess_returns,axis=0)/np.std(excess_returns,axis=0)

In [5]:
pd.concat([df_industries, df_market], axis=1)

,Cnsmr,Manuf,HiTec,Hlth,Other,Market
Date,,,,,,
190101,-1.34,4.20,3.01,-0.71,1.92,1.99
190102,2.89,4.87,3.45,3.33,2.63,3.49
190103,1.89,1.81,-0.83,2.29,-0.86,0.46
190104,4.36,2.58,3.16,6.37,1.11,2.90
190105,0.92,-2.55,-1.13,1.96,-2.40,-1.27
...,...,...,...,...,...,...
190608,1.71,0.80,4.36,2.85,2.59,2.91
190609,-3.28,-2.84,-6.16,-6.01,-2.26,-4.37
190610,8.63,6.01,6.74,2.25,7.25,6.65


In [6]:
performance_df = pd.DataFrame({
    'Sharpe Ratio':       sharpe
    }, index=df.columns)

In [7]:
performance_df

,Sharpe Ratio
Cnsmr,0.468518
Manuf,0.283765
HiTec,0.444596
Hlth,0.406036
Other,0.318934
Market,0.407127


In [8]:
m_model = LinearRegression()
m_model.fit(market_returns, excess_returns)    # only regress Rm-Rf
m_coefficients = m_model.coef_.flatten()                        # CAPM Betas for all industries
m_intercept = m_model.intercept_                                # CAPM Alphas or Jensen Alpha

# calculate metrics
sharpe        = np.mean(excess_returns,axis=0)/np.std(excess_returns,axis=0)
treynor       = np.mean(excess_returns,axis=0)/m_coefficients
jensens_alpha = m_intercept

performance_df = pd.DataFrame({
    'Sharpe Ratio':       sharpe,
    'Treynor Ratio' :     treynor, 
    'Jensen\'s Alpha' :   jensens_alpha, 
    }, index=df.columns)

performance_df

,Sharpe Ratio,Treynor Ratio,Jensen's Alpha
Cnsmr,0.468518,1.803698,3.282538e-01
Manuf,0.283765,1.062580,-3.470519e-01
HiTec,0.444596,1.634133,2.445849e-01
Hlth,0.406036,1.897337,3.843456e-01
Other,0.318934,1.190708,-2.345936e-01
Market,0.407127,1.395139,6.661338e-16




Using market return as target return, calculate and report the information ratio and Sortino ratio for the five industry portfolios.

Briefly explain (in words) the economic significance of using the information ratio and Sortino ratio to measure the performance of well-diversified portfolios.

In [19]:
deviations = df_industries.values - df_market['Market'].values[:, np.newaxis]   # R_i - R_m, 
R = np.mean(deviations,   # and get mean deviation
            axis=0)
tracking_error = np.std(deviations,   # and get mean deviation
            axis=0)

pd.DataFrame(
            {'Mean Monthly excess returns over market %': R,
             'standard deviation': tracking_error},
            index=df_industries.columns    
            )

,Mean Monthly excess returns over market %,standard deviation
Cnsmr,0.054028,1.562249
Manuf,-0.286250,1.582483
HiTec,0.277222,1.362655
Hlth,0.056944,2.560878
Other,-0.028750,1.773860


In [ ]:
info_ratio = R/tracking_error

,Information Ratio
Cnsmr,0.034583
Manuf,-0.180887
HiTec,0.203443
Hlth,0.022236
Other,-0.016208


In [23]:
# calculate downside risk for sortino
below_target_semi_variance = np.sum(np.where(deviations<0,  # filter indices for negative values
                          deviations,    # if false, set value to 0
                          0
                          )**2, 
                          axis=0)/df_industries.count()  

In [26]:

sortino   = np.mean(deviations,axis=0)/ np.sqrt(below_target_semi_variance)

pd.DataFrame(
            {'Information Ratio': info_ratio,
             'Sortino' : sortino},
            index=df_industries.columns    
            )

,Information Ratio,Sortino
Cnsmr,0.034583,0.048502
Manuf,-0.180887,-0.229477
HiTec,0.203443,0.340892
Hlth,0.022236,0.030527
Other,-0.016208,-0.023591


# Question
The data files that you received contain monthly excess returns for five industry portfolios and the market portfolio. Using monthly excess returns for the five industry portfolios and the market portfolio:

Calculate and report the Sharpe ratio and Treynor ratio for the five industry portfolios and the market portfolio.

Using market return as target return, calculate and report the information ratio and Sortino ratio for the five industry portfolios.

Briefly explain (in words) the economic significance of using the information ratio and Sortino ratio to measure the performance of well-diversified portfolios.


---
Answer: 

The information ratio measures a portfolio's excess return relative to a benchmark, adjusted for its tracking error. A higher IR indicates a portfolio's ability to consistently outperform its benchmark with less volatility. This is crucial for investors seeking superior returns while managing risk. Used to rank Portfolio Managers against each other.

Measures ability of fund manager to exceed target return, relative to amount of tracking error. Often used to evaluate fund managers who follow “enhanced indexing” strategy by overweighting or underweighting selected components of stock index


The Sortino ratio focuses on downside risk, measuring a portfolio's return relative to a downside risk threshold. A higher Sortino ratio suggests a portfolio's ability to generate returns while minimizing downside volatility. This is particularly important for risk-averse investors who prioritize downside protection. But these are skewed and does not consider upside returns. 

More informative than information ratio or Sharpe ratio when return distribution is not normal. Because it does not assume normality.